# Внутренние переменные состояния

In [1]:
from typing import TypeAlias

Amount: TypeAlias = float
Balance: TypeAlias = float
BALANCE = 100


def withdraw(amount: Amount) -> Balance:
    global BALANCE
    if BALANCE >= amount:
        BALANCE -= amount
        return BALANCE
    return "Недостаточно денег на счете"

In [2]:
withdraw(25)

75

In [3]:
withdraw(25)

50

In [4]:
withdraw(60)

'Недостаточно денег на счете'

In [5]:
withdraw(15)

35

`balance` это проблема, так как к ней имеет доступ любая функция и может ее изменить.

Сделать `balance` внутренней переменной по отношению к `withdraw` и изменять ее внутри функции.

In [36]:
def new_withdraw() -> Balance:
    balance = Balance(100)
    def helper(amount: Amount) -> Balance:
        nonlocal balance
        if balance >= amount:
            balance -= amount
            return balance
        return "Недостаточно денег на счете"
    return helper

In [37]:
func_withdraw = new_withdraw()
for amount in [25, 25, 60, 35]:
    print(func_withdraw(amount))

75.0
50.0
Недостаточно денег на счете
15.0


Обработчик снятия денег со счета

In [46]:
def make_withdraw(balance: Balance):
    def helper(amount: Amount):
        nonlocal balance
        if balance >= amount:
            balance -= amount
            return balance

        return "Недостаточно денег на счете"
    return helper

In [47]:
w1 = make_withdraw(100)
w2 = make_withdraw(100)

In [48]:
w1(50)

50

In [49]:
w2(70)

30

In [50]:
w2(40)

'Недостаточно денег на счете'

In [51]:
w1(40)

10

w1 и w2 полностью независимые объекты, каждый со своей локальной переменной balance. Снятие денег с одного счета не влияет на другой.

Добавим занесение, чтобы смоделировать простой банковский счет

In [52]:
def make_account(balance: Balance) -> Balance | str:
    def withdraw(amount: Amount) -> Balance | str:
        nonlocal balance
        if balance >= amount:
            balance -= amount
            return balance
        return "Недостаточно денег на счете"

    def deposit(amount: Amount) -> Balance:
        nonlocal balance
        balance += amount
        return balance

    def dispatch(m):
        if m == "withdraw":
            return withdraw
        elif m == "deposit":
            return deposit
        else:
            raise ValueError(f"Неизвестная операция: {m}")

    return dispatch

In [53]:
acc = make_account(100)

In [54]:
acc("withdraw")(50)

50

In [55]:
acc("withdraw")(60)

'Недостаточно денег на счете'

In [56]:
acc("deposit")(40)

90

In [57]:
acc("withdraw")(60)

30